# Oak-D Aruco Tag Distance Test

Use this notebook to test how far away the Oak-d camera can detect an Aruco tag. We've found that a high definition camera (1920x1080) can detect a tag from about 35 feet away, but Oak-d camera has 4k resolution, so hopefully it can sense the tags from a greater distance (the target is 70 feet).

## Setup

You'll need to set up the DepthAI program (see Adrien's setup steps [here](https://github.com/SJSURoboticsTeam/urc-intelligent_systems-2023/blob/main/Vision/README.md)). You will also need to install OpenCV-Contrib version 4.6.0.66 (a lower version is okay, but a higher has breaking API changes).

In [ ]:
!pip install opencv-contrib-python==4.6.0.66

In [1]:
import cv2
import numpy as np
from collections import deque
import os
import glob
import depthai

In [2]:
# load the 4x4_50 aruco tag dictionary
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

## Camera

Here is where we define the camera. NOTE: this code has been modified since the last time I had access to an Oak-D camera, so it hasn't been tested. There may be some subtle semantics error that I don't know about, so if you run into issues either google it or contact me. The important thing is getting the camera initialized for 4k video (not preview, which is more suited for smaller resolutions).

In [3]:
# Pipeline tells DepthAI what operations to perform when running - you define all of the resources used and flows here
pipeline = depthai.Pipeline()

# First, we want the Color camera as the output
cam_rgb = pipeline.createColorCamera()
cam_rgb.setInterleaved(False)
cam_rgb.set_Resolution(depthai.ColorCameraProperties.SensorResolution.THE_4_K)
cam_rgb.setVideoSize(4056, 3040)

# XLinkOut is a "way out" from the device. Any data you want to transfer to host need to be send via XLink
xout_rgb = pipeline.createXLinkOut()
# For the rgb camera output, we want the XLink stream to be named "rgb"
xout_rgb.setStreamName("rgb")
# Linking camera preview to XLink input, so that the frames will be sent to host
cam_rgb.video.link(xout_rgb.input)


!!IMPORTANT!! Before going any further, make sure that the frame is actually a 4k image. If it's not, then play around with things (following this tutorial here: https://docs.luxonis.com/projects/api/en/latest/tutorials/dispaying_detections/) until it works.

In [ ]:
with depthai.Device(pipeline) as device:
    frame = device.getOutputQueue("rgb").get().getCvFrame()
    print(frame.shape)

## Detecting Aruco Tags

This is the main loop where we try to find aruco tags. It'll draw a green bounding box around any tag it detects. It also looks through the last 10 frames to see if it can detect a tag in any of those frames. This helps its accuracy at long distances because the odds of it detecting a tag in single frame isn't great, but over many frames, it's pretty decent. Press 'q' to stop the loop.

In [4]:
frame_queue = deque(maxlen=10)

with depthai.Device(pipeline) as device:
    # From this point, the Device will be in "running" mode and will start sending data via XLink

    # To consume the device results, we get two output queues from the device, with stream names we assigned earlier
    q_rgb = device.getOutputQueue("rgb")

    frame = None

    while True:
        in_rgb = q_rgb.tryGet()

        if in_rgb is not None:
            # If the packet from RGB camera is present, we're retrieving the frame in OpenCV format using getCvFrame
            frame = in_rgb.getCvFrame()
            print(frame.shape)
            frame_queue.append(frame)

        corners, ids = [], []

        for test_frame in frame_queue:
            # detect the aruco markers in the image
            possible_corners, possible_ids, rejectedImgPoints = cv2.aruco.detectMarkers(test_frame, arucoDict,parameters=parameters)
            if len(possible_corners) > 0:
                corners.extend(possible_corners)
                ids.append(possible_ids.flatten())
                break

        for corner, tag_id in zip(corners, ids):
            # loop over the detected ArUCo corners
            #for (markerCorner, markerID) in zip(corners, ids):
            # extract the marker corners (which are always returned in
            # top-left, top-right, bottom-right, and bottom-left order)
            # corners = markerCorner.reshape((4, 2))
            tag_id = tag_id[0]

            (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
            # convert each of the (x, y)-coordinate pairs to integers
            topRight = (int(topRight[0]), int(topRight[1]))
            bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
            bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
            topLeft = (int(topLeft[0]), int(topLeft[1]))
            # draw the bounding box of the ArUCo detection
            cv2.line(frame, topLeft, topRight, (0, 255, 0), 4)
            cv2.line(frame, topRight, bottomRight, (0, 255, 0), 4)
            cv2.line(frame, bottomRight, bottomLeft, (0, 255, 0), 4)
            cv2.line(frame, bottomLeft, topLeft, (0, 255, 0), 4)
            # compute and draw the center (x, y)-coordinates of the ArUco
            # marker
            cX = int((topLeft[0] + bottomRight[0]) / 2.0)
            cY = int((topLeft[1] + bottomRight[1]) / 2.0)
            cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
            # draw the ArUco marker ID on the frame
            cv2.putText(frame, str(tag_id),
                (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (0, 255, 0), 2)

        if frame is not None:
            cv2.resize(frame, None, fx=.25, fy=.25)
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF
            #if the `q` key was pressed, break from the loop
            if key == ord("q"):
               break

RuntimeError: ColorCamera(0) - 'preview' width or height (4056, 3040) bigger than sensor resolution (1920, 1080)

In [ ]:

# close all windows
cv2.destroyAllWindows()